In [ ]:
from models_wg import *
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
from preprocess import train_test_split
from preprocess import n_series
from datetime import datetime, timedelta 
from sklearn.metrics import mean_absolute_error
from collections import OrderedDict
import os
import sys
import csv 
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
def plot(df,kunag,matnr,n,p,d,q,sl,folder):        
        test1 = train_test_split(df,kunag,matnr,n)[1]
        train = train_test_split(df,kunag,matnr,n)[0]
        plt.figure(figsize=(18,10))
        plt.plot(train.set_index("date")['quantity'], label='Train',marker = '.')
        plt.plot(test1.set_index("date")['quantity'], label='Test',marker = '.')
        
        output1,rms1,mae1 = sarima(df,kunag,matnr,n,p,q,d)
        output2,rms2,mae2 = prophet(df, kunag,matnr,n,0.1)
        y_hat_avg1 = output1
        plt.plot(y_hat_avg1.set_index("date")['pred_column'], label='arima011',marker = '.')
        y_hat_avg2 = output2
        plt.plot(y_hat_avg2.set_index("date")['pred_column'], label='prophet',marker = '.')
#         output1,rms1,mae1 = sarima(df,kunag,matnr,n,p,q,d)
        

#         output1,rms1,mae1 = moving_average(df,kunag,matnr,16,21)
#         y_hat_avg1 = output1
#         plt.plot(y_hat_avg1.set_index("date")['pred_column'], label='MA_21',marker = '.')

#         output2,rms2,mae2 = moving_average(df,kunag,matnr,16,23)
#         y_hat_avg2 = output2
#         plt.plot(y_hat_avg2.set_index("date")['pred_column'], label='MA_23',marker = '.')

#         output3,rms3,mae3 = moving_average(df,kunag,matnr,16,25)
#         y_hat_avg3 = output3
#         plt.plot(y_hat_avg3.set_index("date")['pred_column'], label='MA_25',marker = '.')

#         output1,rms1,mae1 = sarima(df,kunag,matnr,n,p,q,d)
#         output2,rms2,mae2 = ses(df,kunag,matnr,n,sl)
#         output3,rms3,mae3 = prophet(df, kunag,matnr,n,sps)
#         output4,rms4,mae4 = croston_tsb(df,kunag,matnr,16,0.2,0.4)

#         y_hat_avg1 = output1
#         plt.plot(y_hat_avg1.set_index("date")['pred_column'], label='SARIMA_' + str(order),marker = '.')
#         y_hat_avg2 = output2
#         plt.plot(y_hat_avg2.set_index("date")['pred_column'], label='SES_' + str(sl),marker = '.')
#         y_hat_avg3 = output3
#         plt.plot(y_hat_avg3.set_index("date")['pred_column'], label='Prophet_' + str(sps),marker = '.',color = "blue")
#         output4,rms4,mae4 = croston_tsb(df,kunag,matnr,16,0.2,0.4)
#         y_hat_avg4 = output4
#         plt.plot(y_hat_avg4.set_index("date")['pred_column'], label='Croston TSM_' +str(0.2)+"_"+str(0.4),marker = '.')
#         output5,rms5,mae5 = croston_tsb(df,kunag,matnr,16,0.2,0.2)
#         y_hat_avg5 = output5
#         plt.plot(y_hat_avg5.set_index("date")['pred_column'], label='Croston TSM_' +str(0.2)+"_"+str(0.2),marker = '.')
#         output6,rms6,mae6 = croston_tsb(df,kunag,matnr,16,0.3,0.3)
#         y_hat_avg6 = output6
#         plt.plot(y_hat_avg6.set_index("date")['pred_column'], label='Croston TSM_' +str(0.3)+"_"+str(0.3),marker = '.')
#         output7,rms7,mae7 = croston_tsb(df,kunag,matnr,16,0.2,0.3)
#         y_hat_avg7 = output7
#         plt.plot(y_hat_avg7.set_index("date")['pred_column'], label='Croston TSM_' +str(0.2)+"_"+str(0.3),marker = '.')
#         output8,rms8,mae8 = croston_tsb(df,kunag,matnr,16,0.3,0.2)
#         y_hat_avg8 = output8
#         plt.plot(y_hat_avg8.set_index("date")['pred_column'], label='Croston TSM_' +str(0.3)+"_"+str(0.2),marker = '.')
        
        
        
        
        
        plt.legend(loc='best')
        plt.title("MA")
        index = str(kunag) + "-" + str(matnr)
        plt.savefig(folder +"/" + 'Graph_{}.png'.format(index), format="PNG")  
        return mae1,mae2
    #         plt.show()

In [ ]:
import os, sys
import time
start = time.time()
bucket = pd.read_csv("bucket.csv")
df = pd.read_csv("4200_C005_2019_03_03.tsv", sep=',', header=None)
df.columns = ["kunag", "matnr", "date", "quantity","price"]
p,d,q = 0,1,1 # arima parameters
sl = 0.2 #smooting coeff for SES
sps = 0.1 # seasonality_prior_scale PROPHET
# a,b = 0.2,0.2 #parameters for croston method
order = (p,q,d)
folder = 'model_graphs/'+"comp(arima_prophet)"
if not os.path.exists(folder):
        os.makedirs(folder)
n = 16
cnt = 0


# main_df = pd.DataFrame(OrderedDict({"kunag" : ["kunag"],"matnr" : ["matnr"],
#                        "sarima" : ["-"],
#                        "ses" : ["-"],
#                        "croston tsb" : "-"}))

# main_df = pd.DataFrame(OrderedDict({"kunag" : ["kunag"],"matnr" : ["matnr"],
#                        "croston tsb0.2_0.4" : "-",
#                        "croston tsb0.2_0.2" : "-",
#                        "croston tsb0.3_0.3" : "-",
#                        "croston tsb0.2_0.3" : "-",
#                        "croston tsb0.3_0.2" : "-"}))

main_df = pd.DataFrame(OrderedDict({"kunag" : ["kunag"],"matnr" : ["matnr"],
                       "arima011" : ["-"],
                       "prophet" : ["-"]}))



for i in range (1,100):
            kunag = int(bucket["kunag"].iloc[i])
            matnr = int(bucket["matnr"].iloc[i])
            index = str(kunag) +"_"+ str(matnr)
            cnt+=1
            print("count",cnt)
            print("index : ",index)
            mae1,mae2 = plot(df,kunag,matnr,n,p,d,q,sl,folder)
            
#             result_df =pd.DataFrame(OrderedDict({"kunag" : [kunag],"matnr" : [matnr],
#                    "sarima" : [round(mae1,3)],
#                    "ses" : [round(mae2,3)],
#                    "croston tsb" : [round(mae4,3)]}))
#             result_df = pd.DataFrame(OrderedDict({"kunag" : ["kunag"],"matnr" : ["matnr"],
#                        "croston tsb0.2_0.4" : [round(mae4,3)],
#                        "croston tsb0.2_0.2" : [round(mae5,3)],
#                        "croston tsb0.3_0.3" : [round(mae6,3)],
#                        "croston tsb0.2_0.3" : [round(mae7,3)],
#                        "croston tsb0.3_0.2" : [round(mae8,3)]}))

            result_df = pd.DataFrame(OrderedDict({"kunag" : [kunag],"matnr" : [matnr],
                       "arima011" : [round(mae1,3)],
                       "prophet" : [round(mae2,3)]}))

            print(result_df)
            main_df = main_df.append(result_df, ignore_index = True)
#             export_csv = main_df.to_csv (r'results/comp8(MA21,23,25).csv',sep = ",", index = None, header=True)
            
            
            
end = time.time()
print("Time Taken : ",end - start)

In [ ]:
main_df
main_df = main_df[1:]
main_df.head()
print(main_df.shape)
data  = main_df
data = data.convert_objects(convert_numeric=True)
round(data.sum(axis = 0, skipna = True),3)

In [ ]:
export_csv = main_df.to_csv (r'results/comp(arima_prophet).csv',sep = ",", index = None, header=True)

In [ ]:
data = pd.read_csv("results/comp(arima_prophet).csv",sep = ",")

In [ ]:
data.head()

In [ ]:
minimum = data.idxmin(axis=1)

In [ ]:
a = 0
s = 0
c = 0
for i in range(0,len(minimum)):
    if(minimum[i]=="arima011"):
        a+=1
    elif(minimum[i]=="prophet"):
        s+=1
#     elif(minimum[i]=="MA_26"):
#         c+=1
print(a)
print(s)
# print(c)

In [ ]:
start = time.time()
bucket = pd.read_csv("bucket.csv")
df = pd.read_csv("4200_C005_2019_03_03.tsv", sep=',', header=None)
df.columns = ["kunag", "matnr", "date", "quantity","price"]


# Path to be created
# path = '/home/rahul/Downloads/bharat/time_series1/arima010'
# if os.path.isfile(path):
#     pass
# else :
#     os.makedirs( path, 493 )

# main_df = pd.DataFrame(OrderedDict({"kunag" : ["-"],
#                     "matnr" : ["-"],
#                     "window8" : ["-"],
#                    "window12" : ["-"],
#                    "window16" : ["-"],
#                    "window20" : ["-"],
#                    "window24" : ["-"],
#                    "window28" : ["-"],
#                    "window30" : ["-"]}))


# m_df = pd.DataFrame({"kunag" : ["kunag"],"matnr" : ["matnr"], "a1" : ["-"],"a2" : ["-"]})

main_df =pd.DataFrame(OrderedDict({"kunag" : ["kunag"],
                    "matnr" : ["matnr"],
                    "window22" : ["-"],
                   "window24" : ["-"],
                   "window26" : ["-"]}))
export_csv= main_df.to_csv (r'results/model_all11.csv',sep = ",", index = None, header=True)

cnt = 0
for i in range (1,5):
            kunag = int(bucket["kunag"].iloc[i])
            matnr = int(bucket["matnr"].iloc[i])
            index = str(kunag) +"_"+ str(matnr)
            print("count",cnt)
            print("index : ",index)
#             print("window :", 4)
#             output1,rms1,mae1 = moving_average(df,kunag,matnr,16,8)
#             output4,rms4,mae4 = ses(df,kunag,matnr,16,0)
#             output2,rms2,mae2 = sarima(df,kunag,matnr,16,0,1,1)

            print("window :", 22)
            output1,rms1,mae1 = moving_average(df,kunag,matnr,16,22)
            print("window :", 24)
            output2,rms2,mae2 = moving_average(df,kunag,matnr,16,24)
            print("window :", 26)
            output3,rms3,mae3 = moving_average(df,kunag,matnr,16,26)
#             print("window :", 20)
#             output4,rms4,mae4 = moving_average(df,kunag,matnr,16,20)
#             print("window :", 24)
#             output5,rms5,mae5 = moving_average(df,kunag,matnr,16,24)
#             print("window :", 28)
#             output6,rms6,mae6 = moving_average(df,kunag,matnr,16,28)
#             print("window :", 30)
#             output7,rms7,mae7 = moving_average(df,kunag,matnr,16,30)
            result_df =pd.DataFrame(OrderedDict({"kunag" : [kunag],
                                "matnr" : [matnr],
                                "window22" : [round(mae1,3)],
                               "window24" : [round(mae2,3)],
                               "window26" : [round(mae3,3)]}))


#             result_df =pd.DataFrame(OrderedDict({"kunag" : [kunag],
#                     "matnr" : [matnr],
#                     "window8" : [round(mae1,3)],
#                    "window12" : [round(mae2,3)],
#                    "window16" : [round(mae3,3)],
#                    "window20" : [round(mae4,3)],
#                    "window24" : [round(mae5,3)],
#                    "window28" : [round(mae6,3)],
#                    "window30" : [round(mae7,3)]}))
                                                 
                                                 
#             result_df = pd.DataFrame({"kunag" : [kunag],"matnr" : [matnr], "a1" : [mae1],"a2" : [mae4]})
#             print("MAE : ",mae1)
#             print(output1)
        
#             print(output2)
            print(result_df["window22"].values[0])
            main_df = main_df.append(result_df, ignore_index = True) 
#             export_csv= main_df.to_csv (r'results/model_all.csv',sep = ",", index = None, header=True)#             print(main_df)
                      
            with open(r'results/model_all11.csv', 'a') as csvfile:
                fieldnames=['kunag','matnr','window22','window24','window26'] 
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#                 writer.writeheader()
                writer.writerow({'kunag' : kunag,'matnr' : matnr,'window22' : result_df["window22"].values[0],'window24' : result_df["window24"].values[0],'window26' : result_df["window26"].values[0]})
#             export_csv= main_df.to_csv (r'results/model_all.csv',sep = ",", index = None, header=True)#             print(main_df)
            cnt+=1
#             if(cnt == 5):                     
#                 break 
# print(main_df)
end = time.time()
print("Time Taken : ",end - start)